In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
# import excel using Data1.xlsx, clear first sheet(夫婦就業) 
# 表１
df = pd.read_excel('data/2_Data.xlsx', index_col=0)
df.head()

,year,設備投資,生産性
id,,,
1,1,10,1
1,2,20,2
2,1,20,2
2,2,40,4
3,1,30,10


In [3]:
# reg 生産性 設備投資, r
# 表２
X = sm.add_constant(df['設備投資'])
Y = df['生産性']
model_sm2 = sm.OLS(Y, X).fit()

# 結果を出力
model_sm2.summary()

/opt/conda/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    生産性   R-squared:                       0.831
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                     39.21
Date:                Tue, 13 Apr 2021   Prob (F-statistic):           0.000243
Time:                        08:31:53   Log-Likelihood:                -32.634
No. Observations:                  10   AIC:                             69.27
Df Residuals:                       8   BIC:                             69.87
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.0000      4.355     -1.607      0.147     -17.043       3.043
設備投資           0.5200      0.083      6.262      0.000       0.328       0.712
==============================================================================
Omnibus:                        0.128   Durbin-Watson:                   2.398
Prob(Omnibus):                  0.938   Jarque-Bera (JB):                0.338
Skew:                          -0.041   Prob(JB):                        0.845
Kurtosis:                       2.103   Cond. No.                         102.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
# xtset id year
# idをカラムに設定
df.reset_index(inplace=True)
# (id, year)をインデックスに設定
gf = df.set_index(['id', 'year'])

# xtreg 生産性 設備投資, fe vce(r)
# 表３
from linearmodels.panel import PanelOLS
import statsmodels.api as sm
exog = sm.add_constant(gf['設備投資'])
grunfeld_fe = PanelOLS(gf['生産性'], exog, entity_effects=True, time_effects=False)
grunfeld_fe = grunfeld_fe.fit()
print(grunfeld_fe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                    生産性   R-squared:                        0.8782
Estimator:                   PanelOLS   R-squared (Between):              0.6429
No. Observations:                  10   R-squared (Within):               0.8782
Date:                Tue, Apr 13 2021   R-squared (Overall):              0.6693
Time:                        08:31:54   Log-likelihood                   -20.048
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      28.845
Entities:                           5   P-value                           0.0058
Avg Obs:                       2.0000   Distribution:                     F(1,4)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             28.845
                            

In [5]:
# xtreg 生産性 設備投資 i.year, fe vce(r)
# 表４

# 2.yearのダミーをgfに追加
dummies = []
for row in gf.iterrows():
    index = row[0]
    if index[1] == 1:
        dummies.append(0)
    else:
        dummies.append(1)
gf['dummies2year'] = dummies

# 計算
exog = sm.add_constant(gf[['設備投資', 'dummies2year']])
grunfeld_fe = PanelOLS(gf['生産性'], exog, entity_effects=True, time_effects=False)
grunfeld_fe = grunfeld_fe.fit()
print(grunfeld_fe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                    生産性   R-squared:                        0.9442
Estimator:                   PanelOLS   R-squared (Between):              0.8496
No. Observations:                  10   R-squared (Within):               0.9442
Date:                Tue, Apr 13 2021   R-squared (Overall):              0.8602
Time:                        08:31:54   Log-likelihood                   -16.150
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      25.358
Entities:                           5   P-value                           0.0132
Avg Obs:                       2.0000   Distribution:                     F(2,3)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             25.358
                            